## Vorverarbeitung

### Packages

In [1]:
# Dokumenten Packages
from IPython.display import display, HTML

import pkg_resources
import random
import numpy as np
import pandas as pd
import missingno as mso
import plotly_express as px
from datetime import datetime


# Packages für Sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

# Linear Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso

# Tree-based Models
from sklearn.tree import DecisionTreeRegressor
# Ensemble Models
from sklearn.ensemble import RandomForestRegressor

In [2]:
import pandas as pd# Pandas Einstellungen
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

# Notebook Einstellungen
display(HTML("<style>.container { width:100% !important; }</style>"))

### Stores und Sales

In [3]:
stores = pd.read_csv('Daten/dmml1_stores.csv')

# Store ID auf Sore_ID ändern
stores.rename(columns={'Store ID':'StoreID'}, inplace=True)

# Ändern zu Datetime von CompetitionOpenSinceYear, CompetitionOpenYear
# stores['CompetitionOpenSinceYea'] = pd.to_datetime(stores['CompetitionOpenSinceYear'], errors='coerce', format='%Y')
# stores['CompetitionOpenYear'] = pd.to_datetime(stores['CompetitionOpenSinceYear'], errors='coerce', format='%Y')
# stores['CompetitionOpenYear'] = stores['CompetitionOpenSinceYear']

# Errechnen des Promotion Start Dates
stores["Promo2SinceWeek"] = stores[stores["Promo2"] == 1]["Promo2SinceWeek"].astype(int).astype(str)
stores["Promo2SinceYear"] = stores[stores["Promo2"] == 1]["Promo2SinceYear"].astype(int).astype(str)
stores["Promo2StartDate"] = pd.to_datetime(stores["Promo2SinceYear"] + stores["Promo2SinceWeek"] + "0", format="%Y%W%w", errors="coerce")

#Drop Columns
stores = stores.drop(
    columns=["Promo2SinceWeek", "Promo2SinceYear", "PromoInterval", 'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth'],
)

# CompetionDistance na Werte zu mean Werte ändern  
stores["CompetitionDistance"] = stores.CompetitionDistance.fillna(stores.CompetitionDistance.mean())

#Ändern von CompetitionDistance bei StoreID, da Annahme falsche Daten (CompetionOpenSince 1900.01.01)
stores.loc[stores['StoreID']==207, 'CompetitionDistance'] = stores.CompetitionDistance.mean()

# Load Sales
sales = pd.read_csv('Daten/dmml1_sales.csv')

#Rename StoreID
sales.rename(columns={'Store ID':'StoreID'}, inplace=True)
#Ändern Date zu datetime
sales['Date'] = pd.to_datetime(sales['Date'], errors='coerce', format='%Y-%m-%d')

#One hot encoding IsHoliday
sales["IsHoliday"] = (sales['StateHoliday'] != "0") | (sales['SchoolHoliday'] != 0)
sales["IsHolidayGeneral"] = (sales['StateHoliday'] == "a")
sales["IsHolidayEaster"] = (sales['StateHoliday'] == "b")
sales["IsHolidayChristmas"] = (sales['StateHoliday'] == "c")
sales["SchoolHoliday"] = (sales['SchoolHoliday'] != 0)

#Drop Columns sales
sales = sales.drop(
    columns=["StateHoliday"]
)

#Sales und Store merge
sales_store = pd.merge(sales, stores, left_on='StoreID', right_on='StoreID', how='inner')

#Promo2RunTime errechnen
sales_store["Promo2RunTime"] = (sales_store["Date"] - sales_store["Promo2StartDate"]).dt.days
sales_store["Promo2RunTime"] = sales_store.Promo2RunTime.fillna(0)

#Drop Columns sales_store 
sales_store = sales_store.drop(
    columns=["Promo2StartDate", 'Customers']
)

#Assortment a,b,c in basic, extra und extended umbennen
assortment_mapping = {'a': 'basic', 'b': 'extra', 'c': 'extended'}
sales_store['Assortment'] = sales_store['Assortment'].replace(assortment_mapping)

#Neues Feature: Monat
sales_store['Month'] = sales_store['Date'].dt.month
sales_store['Month'] = sales_store['Month'].replace({
    1: 'Januar',
    2: 'Februar',
    3: 'März',
    4: 'April',
    5: 'Mai',
    6: 'Juni',
    7: 'Juli',
    8: 'August',
    9: 'September',
    10: 'Oktober',
    11: 'November',
    12: 'Dezember'})

sales_store

,StoreID,DayOfWeek,Date,Sales,Open,Promo,SchoolHoliday,IsHoliday,IsHolidayGeneral,IsHolidayEaster,IsHolidayChristmas,StoreType,Assortment,CompetitionDistance,Promo2,Promo2RunTime,Month
0,1,4,2015-04-30,6884,1,1,False,False,False,False,False,a,basic,570.0,1,1852.0,April
1,1,3,2015-04-29,6764,1,1,False,False,False,False,False,a,basic,570.0,1,1851.0,April
2,1,2,2015-04-28,6861,1,1,False,False,False,False,False,a,basic,570.0,1,1850.0,April
3,1,1,2015-04-27,6523,1,1,False,False,False,False,False,a,basic,570.0,1,1849.0,April
4,1,7,2015-04-26,0,0,0,False,False,False,False,False,a,basic,570.0,1,1848.0,April
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,1,0,False,False,False,False,False,a,extended,9260.0,0,0.0,Januar
246899,300,5,2013-01-04,5524,1,0,True,True,False,False,False,a,extended,9260.0,0,0.0,Januar
246900,300,4,2013-01-03,5563,1,0,True,True,False,False,False,a,extended,9260.0,0,0.0,Januar
246901,300,3,2013-01-02,6218,1,0,True,True,False,False,False,a,extended,9260.0,0,0.0,Januar


## Aufgabe 2 & 3

#### Weitere neue Features 

In [4]:
# Neues Feature "BeforeEaster"
# Beachte: Jedes Jahr liegt Ostern an einem unterschiedlichen Datum

days_before_easter = 7 # In Aufgabe 1 wurde beobachtet, dass meist eine Woche vor Ostern die Umsätze überdurchschnittlich sind

# Neue Spalte "BeforeEaster"
sales_store['BeforeEaster'] = False

# über jedes Jahr iterieren
for year in range(2013, 201):
    # Daten filtern für das aktuelle Jahr
    year_data = sales_store[sales_store['Date'].dt.year == year]
    
    # Datumswerte berechnen für das aktuelle Jahr
    one_week_before_easter = year_data.loc[year_data['IsHolidayEaster'], 'Date'] - timedelta(days=days_before_easter + 1)
    last_day_before_easter = year_data.loc[year_data['IsHolidayEaster'], 'Date'] - timedelta(days=1)
    
    # True für Zeilen im 7-Tage Intervall vor Ostern
    sales_store.loc[sales_store['Date'].between(one_week_before_easter.min(), last_day_before_easter.max()) & (sales_store['Date'].dt.year == year), 'BeforeEaster'] = True




# Features zu Before Christmas: 1 week, 2 weeks, 3 weeks Before Christmas 
# --> Da in Aufgabe 1 sehr Unterschiedliche Umsätze vor Weihnachten beobachtet wurden (höchste Umsätze eine Woche vor Weihnachten)

# Neues Feature "1WeekBeforeChristmas" (16. bis 23. Dezember)
one_week_before_christmas = pd.to_datetime('16.12.', format='%d.%m.')
last_day_before_christmas = pd.to_datetime('23.12.', format='%d.%m.')

# Neue Spalte generieren; True für alle Zeilen wo Date im Inervall 16. bis 23.12 liegt
sales_store['1WeekBeforeChristmas'] = sales_store['Date'].dt.strftime('%m-%d').between(one_week_before_christmas.strftime('%m-%d'), last_day_before_christmas.strftime('%m-%d'))


# Neues Feature "2WeeksBeforeChristmas" (8. bis 15. Dezember)
two_weeks_before_christmas = pd.to_datetime('08.12.', format='%d.%m.')
two_weeks_before_christmas_end = pd.to_datetime('15.12.', format='%d.%m.')

# Neue Spalte generieren; True für alle Zeilen wo Date im Inervall 8. bis 15.12 liegt
sales_store['2WeeksBeforeChristmas'] = sales_store['Date'].dt.strftime('%m-%d').between(two_weeks_before_christmas.strftime('%m-%d'), two_weeks_before_christmas_end.strftime('%m-%d'))


# Neues Feature "3WeeksBeforeChristmas" (1. bis 7. Dezember)
three_weeks_before_christmas = pd.to_datetime('01.12.', format='%d.%m.')
three_weeks_before_christmas_end = pd.to_datetime('07.12.', format='%d.%m.')

# Neue Spalte generieren; True für alle Zeilen wo Date im Inervall 1. bis 7.12 liegt
sales_store['3WeeksBeforeChristmas'] = sales_store['Date'].dt.strftime('%m-%d').between(three_weeks_before_christmas.strftime('%m-%d'), three_weeks_before_christmas_end.strftime('%m-%d'))


# Neues Feature "AfterChristmas" (27. bis 31. Dezember)
after_christmas_start = pd.to_datetime('27.12.', format='%d.%m.')
after_christmas_end = pd.to_datetime('31.12.', format='%d.%m.')

# Neue Spalte generieren; True für alle Zeilen wo Date im Inervall 27. bis 31.12 liegt
sales_store['AfterChristmas'] = sales_store['Date'].dt.strftime('%m-%d').between(after_christmas_start.strftime('%m-%d'), after_christmas_end.strftime('%m-%d'))

## Aufgabe 4 & 5

### Vorbereitung

In [5]:
# Funktion um Gesamtkosten für Aufgabe 5 zu berechnen

def gesamtkosten(y_predicted, y_testdaten):
    print("Aufgabe 5:")

    # Differenzen zwischen vorausgesagtem und echten Wert
    y_differences = y_predicted - y_testdaten

    # in Dataframe konvertieren
    y_differences = y_differences.to_frame(name='Differenzen')

    # Anzahl der vorkommenden starken Unter- und Überschätzungen
    amount_lt_4k = y_differences[y_differences['Differenzen'] < -4000].count().iloc[0]
    amount_bt_3k_6k = y_differences[y_differences['Differenzen'].between(3000, 6000)].count().iloc[0]
    amount_gt_6k = y_differences[y_differences['Differenzen'] > 6000].count().iloc[0]

    print(f"( -4000> : {amount_lt_4k} | 3000-6000 : {amount_bt_3k_6k} | >6000 : {amount_gt_6k}) out of {amount_lt_4k + amount_bt_3k_6k + amount_gt_6k}/{y_differences.count().iloc[0]} - {round(y_differences.count().iloc[0] / (amount_lt_4k + amount_bt_3k_6k + amount_gt_6k), 2)}%")

    # Gesamtkosten durch Unter- und Überschätzungen
    gesamtkosten = 100*amount_lt_4k + 150*amount_bt_3k_6k + 250*amount_gt_6k
    print(f"Es fallen {gesamtkosten}€ Gesamtkosten auf")

In [6]:
X = sales_store
y = sales_store.pop("Sales")

# Split in Trainings- und Testset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Mithilfe von ColumnTransformer:
# 1. Entfernen der Spalte Date
# 2. One-Hot-Encoding der Spalten StoreID, DayOfWeek, StoreTyp, Assortment, Month
# 3. Sklaieren der Spalten CompetitionDistance, Promo2RunTime

column_trans = ColumnTransformer(transformers=[
                                ("Entfernen von Spalten", "drop", ["Date"]),
                                ("One-hot encoding", OneHotEncoder(handle_unknown="ignore"), ["StoreID", "DayOfWeek", "StoreType", "Assortment", "Month"]), 
                                ("Standard", StandardScaler(), ["CompetitionDistance", "Promo2RunTime"])
                                ], remainder="passthrough")
X_transformed = column_trans.fit_transform(X)
feature_names_out = column_trans.get_feature_names_out()

### Entscheidungsbaum: DecisionTreeRegressor 

#### Modelltraining

In [8]:
dt_pipe = Pipeline([("Vorverarbeitung", column_trans), ("dt", DecisionTreeRegressor())])

param_grid = {
    "dt__max_depth": [3, 4, 5],
    #"dt__min_samples_split": [2, 3, 4],
    #"dt__min_samples_leaf": [100, 150, 200],
    #"dt__max_features": [None, "sqrt", "log2"],
    #"Vorverarbeitung__Standard__with_std": [True, False]
}

gs = GridSearchCV(dt_pipe, param_grid, cv=5, error_score='raise')

gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('Vorverarbeitung',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Entfernen '
                                                                         'von '
                                                                         'Spalten',
                                                                         'drop',
                                                                         ['Date']),
                                                                        ('One-hot '
                                                                         'encoding',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['StoreID',
                                                                          'DayOfWeek',
                                                                          'StoreType',
                                                                          'Assortment',
                                                                          'Month']),
                                                                        ('Standard',
                                                                         StandardScaler(),
                                                                         ['CompetitionDistance',
                                                                          'Promo2RunTime'])])),
                                       ('dt', DecisionTreeRegressor())]),
             param_grid={'dt__max_depth': [3, 4, 5]})

#### Beste Parameter und Bewertung

In [19]:
best_params = gs.best_params_

# Bewertung auf den Testdaten
y_pred_dt = gs.predict(X_test)
score = gs.score(X_test, y_test)
mse = mean_squared_error(y_test, y_pred_dt)

print("Beste Parameter:", best_params)
print("Score:", round(score, 4))
print("MSE:", round(mse, 4))

Beste Parameter: {'dt__max_depth': 5}
Score: 0.592
MSE: 6554728.56


#### Feature Importance

In [15]:
feature_importances_dt = gs.best_estimator_['dt'].feature_importances_


feature_importance__dt_df = pd.DataFrame({
    'Feature': feature_names_out,
    'Importance': feature_importances_dt
})


feature_importance__dt_df = feature_importance__dt_df.sort_values(by='Importance', ascending=False)

feature_importance__dt_df.head(5)

,Feature,Importance
328,remainder__Open,0.734602
329,remainder__Promo,0.110197
206,One-hot encoding__StoreID_207,0.065649
61,One-hot encoding__StoreID_62,0.045366
300,One-hot encoding__DayOfWeek_1,0.023182


In [16]:
filtered_dt_df = feature_importance__dt_df[feature_importance__dt_df['Importance'] > 0]

fig = px.histogram(filtered_dt_df, x='Feature', y='Importance', labels={'Importance': 'Feature Importance'}, title='Feature Importance (Values > 0)')
fig.show()

#### Gesamtkosten

In [20]:
gesamtkosten(y_pred_dt, y_test)

Aufgabe 5:
( -4000> : 3256 | 3000-6000 : 3419 | >6000 : 47) out of 6722/49381 - 7.35%
Es fallen 850200€ Gesamtkosten auf


### Ensemble-Modell: RandomForestRegressor

#### Modelltraining

In [22]:
rf_pipe = Pipeline([
    ("Vorverarbeitung", column_trans),
    ("rf", RandomForestRegressor())
])

param_grid_rf = {
    "rf__n_estimators": [3, 4],
    # "rf__max_depth": [None, 10, 20],
    # "rf__min_samples_split": [2, 5, 10],
    # "rf__min_samples_leaf": [1, 2, 4],
    # "rf__max_features": ["sqrt", "log2"]
    
}

gs_rf = GridSearchCV(rf_pipe, param_grid_rf, cv=5,error_score='raise')
gs_rf.fit(X_train, y_train)

KeyboardInterrupt: 

#### Beste Parameter und Bewertung

In [19]:
y_pred_rf = gs_rf.predict(X_test)

# Mittlerer quadratischer Fehler (Mean Squared Error, MSE)
mse = mean_squared_error(y_test, y_pred_rf)

# Bestimmtheitsmaß (R-squared)
r2 = r2_score(y_test, y_pred_rf)

best_params = gs_rf.best_params_
print("Beste Parameter:", best_params)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)


Beste Parameter: {'rf__n_estimators': 4}
Mean Squared Error (MSE): 1141637.7536094566
R-squared (R2): 0.9289434537830707


#### Feature Importance

In [21]:
best_rf_model = gs_rf.best_estimator_['rf']

# Extrahiere die Feature Importance
feature_importances_rf = best_rf_model.feature_importances_

# DataFrame für Feature Importance erstellen
feature_importance__rf_df = pd.DataFrame({
    'Feature': feature_names_out,
    'Importance': feature_importances_rf
})

# Sortiere das DataFrame nach Importance absteigend
feature_importance_rf_df = feature_importance__rf_df.sort_values(by='Importance', ascending=False)
feature_importance_rf_df

NameError: name 'gs_rf' is not defined

In [ ]:
filtered_rf_df = feature_importance_rf_df[feature_importance_rf_df['Importance'] > 0.02]

fig = px.histogram(filtered_rf_df, x='Feature', y='Importance', labels={'Importance': 'Feature Importance'}, title='Feature Importance (Values > 0.02)')
fig.show()

#### Gesamtkosten

In [21]:
gesamtkosten(y_pred_rf, y_test)

Aufgabe 5:
( -4000> : 3256 | 3000-6000 : 3419 | >6000 : 47) out of 6722/49381 - 7.35%
Es fallen 850200€ Gesamtkosten auf


### Lineares Modell: Ridge

#### Modelltraining

In [22]:
# Neue ColumnTransformer, da Ridge und Lasso die Skalierung selber übernehmen 
column_trans = ColumnTransformer(transformers=[
                                ("Entfernen von Spalten", "drop", ["Date"]),
                                ("One-hot encoding", OneHotEncoder(handle_unknown="ignore"), ["StoreID","DayOfWeek", "StoreType", "Assortment", "Month"]),
                                ], remainder="passthrough")
X_transformed = column_trans.fit_transform(X)
feature_names_out = column_trans.get_feature_names_out()

In [23]:
# Ridge Regression Pipeline
ridge_pipe = Pipeline([
    ("Vorverarbeitung", column_trans),
    ("ridge", Ridge())
])

# Grid Search Parameter für Ridge
param_grid_ridge = {
    "ridge__fit_intercept": [False, True],  # Set fit_intercept to False for sparse data
    "ridge__alpha": [0.1, 1.0, 10.0],
    #"ridge__solver": ['auto', 'lsqr']
}

# Grid Search für Ridge und Lasso
gs_ridge = GridSearchCV(ridge_pipe, param_grid_ridge, cv=5,error_score='raise')
gs_ridge.fit(X_train, y_train)


GridSearchCV(cv=2, error_score='raise',
             estimator=Pipeline(steps=[('Vorverarbeitung',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Entfernen '
                                                                         'von '
                                                                         'Spalten',
                                                                         'drop',
                                                                         ['Date']),
                                                                        ('One-hot '
                                                                         'encoding',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['StoreID',
                                                                          'DayOfWeek',
                                                                          'StoreType',
                                                                          'Assortment',
                                                                          'Month'])])),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': [0.1, 1.0, 10.0],
                         'ridge__fit_intercept': [False, True]})

#### Beste Parameter und Bewertung

In [24]:
best_params = gs_ridge.best_params_

# R2-Score
best_r2_score = gs_ridge.best_score_

# MAE und MSE
y_pred_ridge = gs_ridge.predict(X_test)
best_mae = mean_absolute_error(y_test, y_pred_ridge)
best_mse = mean_squared_error(y_test, y_pred_ridge)

# Ausgabe der Ergebnisse
print("Beste Parameter:", best_params)
print("R2-Score:", best_r2_score)
print("MAE:", best_mae)
print("MSE:", best_mse)


Beste Parameter: {'ridge__alpha': 0.1, 'ridge__fit_intercept': True}
R2-Score: 0.8206226771456677
MAE: 1221.7713745557708
MSE: 2941326.3377902256


#### Features Importance

In [25]:
# Beste Ridge-Regression-Instanz aus der GridSearchCV
best_ridge_model = gs_ridge.best_estimator_['ridge']

# Extrahiere die Koeffizienten
ridge_coefficients = best_ridge_model.coef_

# DataFrame für die Koeffizienten erstellen
ridge_coefficients_df = pd.DataFrame({
    'Feature': feature_names_out,
    'Coefficient': ridge_coefficients
})

# Sortiere das DataFrame nach Koeffizientenbetrag absteigend
ridge_coefficients_df = ridge_coefficients_df.reindex(ridge_coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)
ridge_coefficients_df


In [ ]:
fig = px.histogram(ridge_coefficients_df, x='Feature', y='Coefficient', labels={'Coefficient': 'Feature Importance'}, title='Feature Importance (Values > 0.02)')
fig.show()

#### Gesamtkosten

In [26]:
gesamtkosten(y_pred_ridge, y_test)

Aufgabe 5:
( -4000> : 837 | 3000-6000 : 1010 | >6000 : 262) out of 2109/49381 - 23.41%
Es fallen 300700€ Gesamtkosten auf


### Lineares Modell: Lasso 

#### Modelltraining

In [27]:
# Lasso Regression Pipeline
lasso_pipe = Pipeline([("Vorverarbeitung", column_trans), ("lasso", Lasso())])

# Grid Search Parameter für Lasso
param_grid_lasso = {
    #"lasso__fit_intercept": [True, False],
    "lasso__alpha": [0.1, 1.0],
    #"lasso__max_iter": [2000, 3000],
    #"lasso__selection": ['cyclic', 'random'],
    #"lasso__tol": [1e-3, 1e-2]
}

gs_lasso = GridSearchCV(lasso_pipe, param_grid_lasso, cv=5, error_score='raise')
gs_lasso.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
             estimator=Pipeline(steps=[('Vorverarbeitung',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Entfernen '
                                                                         'von '
                                                                         'Spalten',
                                                                         'drop',
                                                                         ['Date']),
                                                                        ('One-hot '
                                                                         'encoding',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['StoreID',
                                                                          'DayOfWeek',
                                                                          'StoreType',
                                                                          'Assortment',
                                                                          'Month'])])),
                                       ('lasso', Lasso())]),
             param_grid={'lasso__alpha': [0.1, 1.0]})

#### Beste Parameter und Berwertung

In [28]:
best_params = gs_lasso.best_params_

# R2-Score
best_r2_score = gs_lasso.best_score_

# MAE und MSE
y_pred_lasso = gs_lasso.predict(X_test)
best_mae = mean_absolute_error(y_test, y_pred_lasso)
best_mse = mean_squared_error(y_test, y_pred_lasso)

# Ausgabe der Ergebnisse
print("Beste Parameter:", best_params)
print("R2-Score:", best_r2_score)
print("MAE:", best_mae)
print("MSE:", best_mse)

Beste Parameter: {'lasso__alpha': 0.1}
R2-Score: 0.8317765829945443
MAE: 1151.2960821084757
MSE: 2727103.70411244


#### Feature Importance

In [29]:
# Beste lasso-Regression-Instanz aus der GridSearchCV
best_lasso_model = gs_lasso.best_estimator_['lasso']

# Extrahiere die Koeffizienten
lasso_coefficients = best_lasso_model.coef_

# DataFrame für die Koeffizienten erstellen
lasso_coefficients_df = pd.DataFrame({
    'Feature': feature_names_out,
    'Coefficient': lasso_coefficients
})

# Sortiere das DataFrame nach Koeffizientenbetrag absteigend
lasso_coefficients_df = lasso_coefficients_df.reindex(lasso_coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)
lasso_coefficients_df

In [ ]:
filtered_lasso_df = lasso_coefficients_df[lasso_coefficients_df['Coefficient'] > 0]
fig = px.histogram(filtered_lasso_df, x='Feature', y='Coefficient', labels={'Coefficient': 'Feature Importance'}, title='Feature Importance (Values > 0.0)')
fig.show()

#### Gesamtkosten

In [30]:
gesamtkosten(y_pred_lasso, y_test)

Aufgabe 5:
( -4000> : 743 | 3000-6000 : 707 | >6000 : 294) out of 1744/49381 - 28.31%
Es fallen 253850€ Gesamtkosten auf


### Fazit

Das Ensemble Modell (der Random Forest Regressor) schneidet im Vergleich zu den anderen Modellen am besten ab, wenn man nur auf den score achtet. Aber bei Lasso sind die Gesamtkosten deutlich niedriger.